# YOLOv8 Pose Estimation Proof of Concept

This notebook demonstrates the setup and execution of a pose estimation project using YOLOv8 in the Chickenvision project. The steps include downloading necessary assets from a GitHub release, setting up the environment, running pose estimation on a video file, and analyzing the results.

## Prerequisites

Before running this notebook, ensure you have the following:
- Anaconda installed
- A GitHub personal access token (PAT) for downloading assets from a private repository

1. **Install Required Libraries:**
   First, you need to install the required libraries such as `ultralytics` for YOLOv8, `torch`, `opencv-python`, `requests`, and `matplotlib`.

In [ ]:
%pip install ultralytics
%pip install torch
%pip install torch torchvision torchaudio
%pip install opencv-python
%pip install opencv-python-headless
%pip install requests
%pip install matplotlib

2. **Download and Prepare the Video Source:**
   If you have a video file, you can upload it manually to your Colab environment or download it from a URL.

In [ ]:
# Example of downloading a video file from a URL
!wget -O ./content/sample_video.mp4 'https://assets.mixkit.co/videos/preview/mixkit-politician-escaping-from-journalists-23111-large.mp4'

3. **Set Up the Directory Structure:**
   Create directories for storing model files and output files.

In [ ]:
!mkdir -p ./content/models
!mkdir -p ./content/outputs

In [ ]:
import os
import csv
import json
import math
import cv2
import torch
import numpy as np
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm
from ultralytics import YOLO

## Constants

In [ ]:

GITHUB_PAT = "YOUR_GITHUB_PAT"  # Replace with your GitHub Personal Access Token
REPO = "projectPavoculus/chickenvision-server"
RELEASE_TAG = "v1.0.1"
MODEL_FILENAMES = ["yolov8n-pose.pt", "yolov8n-seg.pt", "yolov8n.pt"]
DOWNLOAD_DIR = "./content/models"
OUTPUT_CSV = "./content/keypoints_output.csv"
VIDEO_SOURCE = "./content/sample_video.mp4"  # Replace with your video file

## Python Script for Pose Estimation

Downloads model files from the specified GitHub release using the GitHub API.

In [ ]:
def download_model_files():
    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)

    base_url = f"https://api.github.com/repos/{REPO}/releases/tags/{RELEASE_TAG}"
    response = requests.get(base_url, headers={"Authorization": f"token {GITHUB_PAT}"})
    release_data = response.json()

    for asset in release_data.get("assets", []):
        if asset["name"] in MODEL_FILENAMES:
            download_url = asset["browser_download_url"]
            download_path = os.path.join(DOWNLOAD_DIR, asset["name"])
            print(f"Downloading {asset['name']}...")
            with requests.get(download_url, headers={"Authorization": f"token {GITHUB_PAT}"}, stream=True) as r:
                r.raise_for_status()
                with open(download_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Downloaded {asset['name']} to {download_path}")

Parses the keypoints from the given CSV file.

In [ ]:
def parse_keypoints_csv(file_path):
    keypoints = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            frame, person, keypoint, x, y = row
            keypoints.append((int(frame), int(person), int(keypoint), float(x), float(y)))
    return keypoints

Counts the number of lines in a file.

In [ ]:
def count_lines(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file)

Saves the keypoints from the YOLO model results to a CSV file.

In [ ]:
def save_keypoints(results, output_file):
    with open(output_file, 'w', newline='') as file:
        if count_lines(output_file) > 2:
            raise Exception("The file already has the necessary.")
        writer = csv.writer(file)
        writer.writerow(["Frame", "Person", "Keypoint", "X", "Y"])
        for i, result in enumerate(results):
            keypoints = result.keypoints.xy[0]
            for j, person_keypoints in enumerate(keypoints):
                for k, keypoint in enumerate(person_keypoints):
                    try:
                        x, y = person_keypoints[:2]
                        writer.writerow([i, j, k, x.item(), y.item()])
                    except IndexError as e:
                        print(f"Error accessing keypoint: {e}")
                        print(f"Keypoint data: {person_keypoints}")


Separates keypoints by frame and person.

In [ ]:
def separate_keypoints(keypoints):
    separated_keypoints = {}
    for frame, person, keypoint, x, y in keypoints:
        if frame not in separated_keypoints:
            separated_keypoints[frame] = {}
        if person not in separated_keypoints[frame]:
            separated_keypoints[frame][person] = []
        separated_keypoints[frame][person].append((keypoint, x, y))
    return separated_keypoints

Generates a graph of keypoints for a given frame.

In [ ]:
def generate_graph(frame_keypoints, frame_idx):
    plt.figure(figsize=(10, 6))
    colors = plt.cm.get_cmap('tab10', len(frame_keypoints[frame_idx]))

    for person_id, keypoints_list in frame_keypoints[frame_idx].items():
        x = [kp[1] for kp in keypoints_list]
        y = [kp[2] for kp in keypoints_list]
        plt.scatter(x, y, color=colors(person_id), label=f"Person {person_id}")
        for i, (xi, yi) in enumerate(zip(x, y)):
            plt.annotate(str(i), (xi, yi))

    plt.gca().invert_yaxis()
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title(f'Frame {frame_idx} Keypoints')
    plt.legend()
    plt.show()

Displays keypoints with optional graph generation.

In [ ]:
def display_keypoints(keypoints, save_graph=False):
    for frame_idx, frame_keypoints in keypoints.items():
        print(f"Frame {frame_idx}:")
        if save_graph:
            generate_graph(keypoints, frame_idx)
        for person_id, keypoints_list in frame_keypoints.items():
            print(f"Person {person_id}:")
            for keypoint, x, y in keypoints_list[:6]:
                print(f"Keypoint {keypoint}: (x={x}, y={y})")
            print()

Main function to run the pose estimation.

In [ ]:
def main():
    # Download model files
    download_model_files()

    # Initialize the model
    model_path = os.path.join(DOWNLOAD_DIR, "yolov8n-pose.pt")
    model = YOLO(model_path)

    # Run prediction
    print("Running pose estimation...")
    results = model.predict(source=VIDEO_SOURCE, save=True, conf=0.5, save_txt=False, show=True)

    # Save keypoints to CSV
    save_keypoints(results, OUTPUT_CSV)

    # Parse and display keypoints
    parsed_keypoints = parse_keypoints_csv(OUTPUT_CSV)
    separated_keypoints = separate_keypoints(parsed_keypoints)
    # display_keypoints(separated_keypoints, save_graph=True)

### Instructions
1. Open Google Colab and create a new notebook.
2. Copy the provided code block into a cell.
3. Replace `GITHUB_PAT = "YOUR_GITHUB_PAT"` with your actual GitHub Personal Access Token.
4. Replace the `VIDEO_SOURCE` URL with the URL to your video file.
5. Run the cell to execute the script.

This notebook cell installs the necessary libraries, sets up the directory structure, downloads the video and model files, and runs the pose estimation script. The script will display keypoints on the video frames and generate graphs for visualization.

In [ ]:
if __name__ == "__main__":
    main()